In [ ]:
filename = 'points_NSXm_TMD_25_16_11_2.json'
In = 11.2


import numpy as np
import json

def get_relay_curve(In=1, order=4, C=1):
    def relay_curve(I, *ks):
        t = 0
        for order, k in enumerate(ks):
            t += k*np.log10((I/In)-C)**order
        return 10**t
    return relay_curve

with open(filename) as f:
    data = json.load(f)['Points']

x = np.array([row['X'] for row in data])/In
y = np.array([row['Y'] for row in data])
points_n = np.argwhere(x == np.min(x))[0,0]
i_l = x[3:points_n][::-1]
t_l = y[3:points_n][::-1]
i_u = x[points_n+3:]
t_u = y[points_n+3:]

l_filter = t_l <= 3600*2
u_filter = t_u <= 3600*2
i_l = i_l[l_filter]
t_l = t_l[l_filter]
i_u = i_u[u_filter]
t_u = t_u[u_filter]

def reduce(i, t):
    i1 = i[0]
    i2 = i[-1]
    i_n = np.geomspace(i1,i2,num=50)
    i_n_nd = np.tile(i_n, (len(i), 1))
    i_nd = np.tile(i, (len(i_n), 1)).T
    idx = (np.abs(i_n_nd - i_nd)).argmin(axis=0)
    return i[idx], t[idx]

i_u, t_u = reduce(i_u, t_u)
i_l, t_l = reduce(i_l, t_l)

In [18]:
import matplotlib.pyplot as plt
%matplotlib auto

plt.plot(i_u, t_u, 'b-', label='upper')
plt.plot(i_l, t_l, 'r-', label='lower')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1,100)
plt.ylim(0.1,1e4)
plt.grid(True, which='major', alpha=1)
plt.grid(True, which='minor', alpha=0.5)
plt.legend()

Using matplotlib backend: TkAgg


In [37]:
# Curve fitting upper

In = 1.28
order=6
relay_curve = get_relay_curve(In, order)

xdata = np.array(i_u)
ydata = np.array(t_u)

xdata_mod = np.log10((xdata/In) -1)
ydata_mod = np.log10(ydata)
coeff = np.polyfit(xdata_mod, ydata_mod, order)[::-1]

plt.plot(xdata, ydata, 'b-', label='data')
x = np.geomspace(min(xdata), 45, num=100)
plt.plot(x, relay_curve(x, *list(coeff)), 'r-', label='fit')
plt.xscale('log')
plt.yscale('log')
plt.grid(True, which='major', alpha=1)
plt.grid(True, which='minor', alpha=0.5)
plt.legend()
display(list(np.round(coeff,5)))

[2.2561, -1.81685, -0.45015, 0.48887, 0.30303, -0.0952, -0.07088]

In [40]:
# Curve fitting lower

In=1.03
order=7
relay_curve = get_relay_curve(In, order)

xdata = np.array(i_l)
ydata = np.array(t_l)

xdata_mod = np.log10((xdata/In) -1)
ydata_mod = np.log10(ydata)
coeff = np.polyfit(xdata_mod, ydata_mod, order)[::-1]

plt.plot(xdata, ydata, 'b-', label='data')
x = np.geomspace(min(xdata), 30, num=100)
plt.plot(x, relay_curve(x, *list(coeff)), 'r-', label='fit')
plt.xscale('log')
plt.yscale('log')
plt.grid(True, which='major', alpha=1)
plt.grid(True, which='minor', alpha=0.5)
plt.legend()
display(list(np.round(coeff,5)))

[1.51107, -1.76534, -0.20344, 0.5632, 0.25239, -0.20003, -0.05773, 0.01754]

In [33]:
print(np.min(i_l), np.min(i_u))
print(json.dumps(list(np.round(i_u, 5))), '\n')
print(json.dumps(list(np.round(t_u, 5))), '\n')
print(json.dumps(list(np.round(i_l, 5))), '\n')
print(json.dumps(list(np.round(t_l, 5))), '\n')

1.0362500000000001 1.305000000000004
[1.305, 1.405, 1.505, 1.62375, 1.7425, 1.87375, 2.01125, 2.16125, 2.32375, 2.49875, 2.68625, 2.88625, 3.105, 3.33625, 3.58625, 3.855, 4.1425, 4.455, 4.78625, 5.14875, 5.53625, 5.94875, 6.38, 6.88, 7.38, 7.9425, 8.505, 9.1925, 9.88, 10.63, 11.38, 12.255, 13.1925, 14.13, 15.1925, 16.38, 17.5675, 18.88, 20.3175, 21.8175, 23.4425, 25.1925, 27.13, 29.13, 31.3175, 33.6925, 36.1925, 38.88, 41.8175, 44.9425] 

[6498.68736, 2938.43071, 1910.10088, 1212.66868, 857.64354, 623.38067, 456.71576, 337.73767, 260.18588, 200.36026, 154.43101, 119.26336, 94.00096, 75.45314, 60.48621, 48.48749, 38.90889, 32.20945, 26.86562, 22.33558, 18.59022, 15.54792, 13.51009, 11.61131, 10.08605, 8.7033, 7.64136, 6.80388, 6.10915, 5.65677, 5.29688, 4.93176, 4.59341, 4.29921, 4.00894, 3.72834, 3.48505, 3.25115, 3.02908, 2.82803, 2.63878, 2.46182, 2.29208, 2.14016, 1.99584, 1.86002, 1.73599, 1.62014, 1.51027, 1.40889] 

[1.03625, 1.11125, 1.18625, 1.27375, 1.36125, 1.46125, 1.56125, 